In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from datetime import date

In [2]:
# Carrega os dados do histórico de chamadas
df = pd.read_csv(r'D:\callback_predictor_machine_learning\data\raw\historico_rechamada.csv')

# Remove linhas com valores ausentes
df = df.dropna()

# Converte a coluna de data para o tipo datetime
df['data'] = pd.to_datetime(df['data'])

In [4]:
# Define as colunas de entrada (features) e o alvo (target)
features = [
    'num_contrato', 'num_cliente', 'cd_skill', 'passo_chamada',
    'turno', 'veterano_novato', 'motivo_script', 'submotivo_script'
]
target = 'rechamada_72h'

# Converte variáveis categóricas para numéricas usando LabelEncoder
label_encoders = {}
for col in features:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

In [5]:
# Define a data atual (pode ser fixada ou automatizada com date.today())
dia_atual = pd.to_datetime('2025-07-29').date()
# dia_atual = date.today()  # Alternativa dinâmica

# Separa os dados do dia atual e os históricos
df_dia_atual = df[df['data'].dt.date == dia_atual].copy()
df_historico = df[df['data'].dt.date < dia_atual].copy()

In [6]:
# Separa as features e o target para os dados históricos
X = df_historico[features]
y = df_historico[target]

# Divide os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treina o modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
# Avalia o modelo com os dados de teste
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')

Acurácia do modelo: 99.30%


In [8]:
# Faz previsões para os dados do dia atual de vez model.predict(X_dia_atual)
if df_dia_atual.empty:
    print(f"Nenhum dado encontrado para o dia {dia_atual}.")
else:
    X_dia_atual = df_dia_atual[features]
    y_dia_atual_pred = model.predict(X_dia_atual)

    # Exibe as previsões para os primeiros 10 contratos
    for idx, pred in enumerate(y_dia_atual_pred):
        num_contrato = df_dia_atual.iloc[idx]['num_contrato']
        previsao = "Rechamada" if pred == 1 else "Sem rechamada"
        print(f'Contrato {num_contrato} - Previsão: {previsao}')
        if idx == 10:
            break

Contrato 818315 - Previsão: Rechamada
Contrato 655839 - Previsão: Rechamada
Contrato 168148 - Previsão: Rechamada
Contrato 778843 - Previsão: Sem rechamada
Contrato 996942 - Previsão: Sem rechamada
Contrato 371836 - Previsão: Rechamada
Contrato 336584 - Previsão: Sem rechamada
Contrato 364712 - Previsão: Sem rechamada
Contrato 475713 - Previsão: Sem rechamada
Contrato 556551 - Previsão: Sem rechamada
Contrato 301664 - Previsão: Sem rechamada
